<a href="https://colab.research.google.com/github/harshtrivedi07/Consequences_covid_virus/blob/main/Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Building an Image Classifier from Scratch

The main goal of this lab is to create an image classifier from scratch using the Keras package. You will create a binary classifier based on an object type of your choice. The classifier will accept images and then predict whether or not a given image contains that object. For example, if you chose your object to be a cat, then the classifier would accept images and predict 1 if it believes there is a cat in the image or a 0 if not. 

You will build and train a classifier in two ways: 
- first using the original data set that you create, and 
- second, using data augmentation techniques. 

**Grading:** 
**Grading:** 

 - 50% of the grade will come from FINAL, error-free code written in Python/Keras that accomplishes all the steps outlined in the instructions for each part of this lab and written in Python/Keras 
 - 50% will come from descriptive comments associated with that code, where the comments explain what the code is doing and why it is important to the overall objective. Thus, comments like "split the data" or "train the model" would receive a grade of 0 as they do not indicate any understanding.

**Research Required:** 

To complete this lab successfully you will need to do some research. At the very least, you will need to implement the [`ImageDataGenerator` class](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) and the [`.flow_from_directory` method](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory). 

**What to submit:**
- A copy of this notebook with:
    - Error-free code in Python/Keras
    - All code cells executed and output visible
- Include a zipped file of your images



## Part 1: Prep Work
### Choose an Object

**You may choose any object other than a cat**.

### Create a Dataset
Use the code provided below to:
 - download a minimum of 500 **useable** images of the chosen object
 - download a minimum of 500 **useable** images of other things that are not your object; there should be **at least 10 different types of objects** in this group of images
 
You should manually verify that the images are indeed of the correct object; that is, sometimes the code will download images that are not of the desired search term and should be deleted. 
 
From these images you will create a training folder (minimum of 350 object/350 not object), a validation folder (minimum of 50 object/50 not object), and a test folder (minimum of 100 object/100 not object). The final counts should approximately represent a 70%/10%/20% split. **Code to create the training, validation, and test folders should be included.**

It's best if your dataset is diverse, so verify that the images of the chosen object have, for example, different lighting, are from different angles, different distances from camera. Similarly for the *not object* images: try to include a variety of objects in different ways. 

**The images downloaded should not be used for any purpose other than personal academic use unless noted otherwise by the copyright holder of the image.**

#### Code for image download

Use the code cells below to download the required images. Note that you will most likely need to request more images than the required amount as some images will not be useable. 

In [ ]:
# modified version of code at https://python.plainenglish.io/how-to-automatically-download-bulk-images-for-your-dataset-using-python-f1efffba7a03
import os
import requests
import pathlib
from bs4 import BeautifulSoup

In [ ]:
google_image = "https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&"

user_agent = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
}

In [ ]:
def download_images():
    data = input('What are you looking for? ')
    n_images = int(input('How many images do you want? '))

    print(f'\nSearching for {n_images}...\n')

    search_url = google_image + 'q=' + data

    response = requests.get(search_url, headers=user_agent)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    results = soup.findAll('img', {'class': 'rg_i Q4LuWd'})

    count = 1
    links = []
    
    for result in results:
        try:
            link = result['data-src']
            links.append(link)
            count += 1
            if(count > n_images):
                break

        except KeyError:
            continue
            
    print(f"Found {len(links)} images.\n")
    print(f"Downloading {len(links)} images...\n")

    curr_path = pathlib.Path()
    saved_folder = curr_path / 'images' / data

    pathlib.Path(saved_folder).mkdir(parents=True, exist_ok=True)

    for i, link in enumerate(links):
        response = requests.get(link)
        filename = data + str(i+1) + '.jpg'
        image_name = saved_folder / filename

        with open(image_name, 'wb') as fh:
            fh.write(response.content)
    
    print(f"Finished downloading!")


In [6]:
download_images()

What are you looking for? cat
How many images do you want? 95

Searching for 95...

Found 80 images.


Finished downloading!


In [7]:
import pathlib
path = pathlib.Path().cwd()
print(path)
images_path = path/"images"
print(images_path) 

dict = {}

for fldr in images_path.iterdir():
  print(fldr.name)
  dict[fldr.name] = []
for img in fldr.iterdir():
   dict[fldr.name].append(img)
  
dict

/content
/content/images
cat


{'cat': [PosixPath('/content/images/cat/cat68.jpg'),
  PosixPath('/content/images/cat/cat58.jpg'),
  PosixPath('/content/images/cat/cat28.jpg'),
  PosixPath('/content/images/cat/cat3.jpg'),
  PosixPath('/content/images/cat/cat45.jpg'),
  PosixPath('/content/images/cat/cat22.jpg'),
  PosixPath('/content/images/cat/cat12.jpg'),
  PosixPath('/content/images/cat/cat50.jpg'),
  PosixPath('/content/images/cat/cat75.jpg'),
  PosixPath('/content/images/cat/cat57.jpg'),
  PosixPath('/content/images/cat/cat2.jpg'),
  PosixPath('/content/images/cat/cat4.jpg'),
  PosixPath('/content/images/cat/cat62.jpg'),
  PosixPath('/content/images/cat/cat46.jpg'),
  PosixPath('/content/images/cat/cat55.jpg'),
  PosixPath('/content/images/cat/cat24.jpg'),
  PosixPath('/content/images/cat/cat25.jpg'),
  PosixPath('/content/images/cat/cat36.jpg'),
  PosixPath('/content/images/cat/cat8.jpg'),
  PosixPath('/content/images/cat/cat5.jpg'),
  PosixPath('/content/images/cat/cat71.jpg'),
  PosixPath('/content/images/cat

In [8]:
dict.keys()

dict_keys(['cat'])

In [9]:
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

In [10]:
x_train=[]
for fldr in images_path.iterdir():
  print(fldr.name)
for img in fldr.iterdir():
    img = load_img(img)
    img_array = img_to_array(img)
    img_array_resized = smart_resize(img_array, [224,224])
    x_train.append(img_array_resized) 

        
x_train=np.array(x_train)
       


cat


In [11]:
x_train.shape

(80, 224, 224, 3)

## Part 2: Using the Original Data

### Dataset Verification

You should verify that the training, validation, and test sets have the correct number of images (the `os` functions should help with this) and that the data is organized in `train`, `validation`, and `test` folders. 

### Dataset Loading and Processing with Image Data Generators

You should now create image data generators for your training, validation, and test datasets that will allow you to feed them into your model in batches. The images should be resized to 180x180 pixels and scaled so all pixel values are between 0 and 1. 

### Create and Evaluate a Model

Build a convolutional neural network and use the validation loss and accuracy to select the best architecture and hyperparameters so that you can maximize the validation accuracy. 

Any overfitting should be addressed; that is, if your model begins overfitting after epoch 3, you should not quote validation accuracy after epoch 15. And, appropriate attempts should be made to reduce/eliminate the overfitting to improve overall model generalizability. 

### Report Accuracy on Test Set 

Your final step is to make predictions using the test set and report the final test set accuracy. 

## Part 3: Using Data Augmentation

### Dataset Loading and Processing with Create Image Data Generators

You should now create image data generators for your training, validation, and test datasets that will allow you to feed them into your model in batches. The images should be resized to 148x148 pixels and scaled so all pixel values are between 0 and 1. 

### Add Data Augmentation 

You should now add data augmentation to your training data generator ONLY. All useful types of augmentation should be used. 

### Create and Evaluate a Baseline Model

Use the best model from **Part 2** to train with the augmented data and evaluate on the validation data to get a baseline accuracy for the new model trained on the augmented dataset. 

### Tune the Model

With a more complex training set, you may be able to improve the accuracy through adjustments to the baseline model architecture and tuning the hyperparameters. 

### Report Accuracy on Test Set 

Your final step is to make predictions using the test set and report the final test set accuracy. 